In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.animation import FFMpegWriter

In [ ]:
# Output directory
export_dir = Path('ff_visuals')
export_dir.mkdir(parents=True, exist_ok=True)

plt.rcParams['figure.dpi'] = 150
LINEWIDTH = 5.0  # thick for presentations
PASTEL_COLORS = ['#BE6427', '#7EAB55', '#85C6D9']  # soft pink, mint, lavender
BLACK = '#000000'
FIGSIZE = (10, 3.5)  # wide and short (presentation friendly)


In [ ]:
def make_clean_ax(fig=None):
    if fig is None:
        fig = plt.figure(figsize=FIGSIZE)
    ax = fig.add_subplot(111)
    ax.set_axis_off()
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(-2.5, 2.5)
    return fig, ax

def save_svg(fig, fname):
    path = export_dir / fname
    
    # FIX: Add invisible points at the exact corners of the view limits.
    # This prevents bbox_inches='tight' from cropping out empty space,
    # ensuring that all SVGs share the exact same coordinate system scaling.
    ax = fig.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.scatter([xlim[0], xlim[1], xlim[0], xlim[1]], 
               [ylim[0], ylim[0], ylim[1], ylim[1]], 
               s=0, alpha=0)  # Invisible corner markers
    
    fig.savefig(path, format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
    plt.close(fig)
    return path

def draw_triangle_impulse(ax, xpos, height, color, base_width=0.08):
    # Draw a filled triangle pointing upward centered at xpos
    verts = np.array([[xpos-base_width/2, 0.0],
                      [xpos+base_width/2, 0.0],
                      [xpos, height]])
    poly = Polygon(verts, closed=True, facecolor=color, edgecolor='none', alpha=0.95)
    ax.add_patch(poly)
    # draw a thin stem for ground connection
    ax.plot([xpos, xpos], [0, height*0.95], linewidth=LINEWIDTH*0.25, color=color, alpha=0.9, solid_capstyle='round')


In [ ]:
# Create domain
x = np.linspace(-2, 2, 4000)

# Frequencies (user suggested maybe multiply with pi)
omega = np.pi * np.array([0.5, 1.2, 1.9])  # angular frequencies (radians per unit)
signals = [np.sin(w * x) for w in omega]
sum_signal = np.sum(signals, axis=0)

In [ ]:
# ---------------- Plot 1: individual sinusoids (pastel) and their sum (black) ----------------
# individual sinusoids
for idx, (s, c) in enumerate(zip(signals, PASTEL_COLORS)):
    fig, ax = make_clean_ax()
    ax.plot(x, s, linewidth=LINEWIDTH, color=c, solid_capstyle='round', alpha=0.9)
    path1 = save_svg(fig, f'signal_components_{idx}.svg')

# sum in black (on top)
fig, ax = make_clean_ax()
ax.plot(x, sum_signal, linewidth=LINEWIDTH*1.1, color=BLACK, solid_capstyle='round', alpha=1.0)
path1 = save_svg(fig, 'signal_components_sum.svg')


In [ ]:

# ---------------- Plot 2: Fourier domain Dirac deltas (impulses) ----------------
# We'll represent impulses at +/-omega as triangular markers. Choose frequency axis in radians.
freq_axis = np.linspace(-np.pi*2.2, np.pi*2.2, 800)
fig = plt.figure(figsize=(8,3))
ax = fig.add_subplot(111)
ax.set_axis_off()
ax.set_xlim(freq_axis.min(), freq_axis.max())
ax.set_ylim(-0.1, 1.1)
# Draw impulses
max_h = 1.0
for w, c in zip(omega, PASTEL_COLORS):
    draw_triangle_impulse(ax, +w, max_h, c, base_width=0.12)
    draw_triangle_impulse(ax, -w, max_h, c, base_width=0.12)
path2 = save_svg(fig, 'spectrum_diracs.svg')

In [ ]:


# ---------------- Plot 3: Gaussian filter in spatial domain (black) ----------------
sigma = 0.25
g_spatial = np.exp(-0.5 * (x / sigma)**2)
# normalize to peak 1
g_spatial = g_spatial / g_spatial.max()
fig, ax = make_clean_ax()
ax.plot(x, g_spatial, linewidth=LINEWIDTH*1.1, color=BLACK, solid_capstyle='round')
ax.set_ylim(-0.05, 1.05)
path3 = save_svg(fig, 'gaussian_spatial.svg')


In [ ]:
# ---------------- Plot 4 (FIXED): Gaussian in frequency domain ----------------
# We remove the * 1.7 factor so the x-axis matches Plot 2 exactly.
freq_axis = np.linspace(-np.pi*2.2, np.pi*2.2, 800)
w = freq_axis.copy() # We redefine 'w' here to be the correct standard range

g_freq = np.exp(-0.5 * (sigma * w)**2)
g_freq = g_freq / g_freq.max() # Normalize peak to 1

fig = plt.figure(figsize=(8,3))
ax = fig.add_subplot(111)
ax.set_axis_off()
ax.set_xlim(w.min(), w.max())
ax.set_ylim(-0.05, 1.05)
ax.plot(w, g_freq, linewidth=LINEWIDTH*1.1, color=BLACK, solid_capstyle='round')

path4 = save_svg(fig, 'gaussian_frequency.svg')

In [ ]:
fig = plt.figure(figsize=(8,3))
ax = fig.add_subplot(111)
ax.set_axis_off()
ax.set_xlim(w.min(), w.max()) # Matches Plot 4 and Plot 2
ax.set_ylim(-0.1, 1.1)        # Matches Plot 2

for wi, c in zip(omega, PASTEL_COLORS):
    # Calculate attenuation
    amp = np.exp(-0.5 * (sigma * wi)**2)
    
    # Draw impulses
    draw_triangle_impulse(ax, +wi, amp, c, base_width=0.12)
    draw_triangle_impulse(ax, -wi, amp, c, base_width=0.12)

path5 = save_svg(fig, 'attenuated_diracs_fixed.svg')

In [ ]:
# ---------------- Plot 6: Inverse transform -> filtered sinusoids + sum ----------------
# Since the original signal is a sum of sinusoids, filtering in freq domain simply scales each sinusoid amplitude.
scales = np.exp(-0.5 * (sigma * omega)**2)
filtered_signals = [s * scale for s, scale in zip(signals, scales)]
filtered_sum = np.sum(filtered_signals, axis=0)

for idx, (s, c) in enumerate(zip(filtered_signals, PASTEL_COLORS)):
    fig, ax = make_clean_ax()
    ax.plot(x, s, linewidth=LINEWIDTH, color=c, solid_capstyle='round', alpha=0.9)
    path6 = save_svg(fig, f'filtered_signal_{idx}.svg')

fig, ax = make_clean_ax()
ax.plot(x, filtered_sum, linewidth=LINEWIDTH*1.1, color=BLACK, solid_capstyle='round')
ax.set_ylim(-2.2, 2.2)
path6 = save_svg(fig, 'filtered_signal_sum.svg')


In [ ]:
# %%
# ---------------- Corrected Animation: Convolution ----------------

# 1. Define a helper to evaluate the signal anywhere (infinite domain)
#    This prevents the "drop off" at the edges by allowing us to integrate
#    beyond x=[-2, 2].
def get_signal_val(x_in):
    # Re-calculate sinusoids at these specific x coordinates
    return np.sum([np.sin(w * x_in) for w in omega], axis=0)

# 2. Precompute convolution properly
shifts = np.linspace(-2.0, 2.0, 240)
conv_vals = []

# Normalization factor ensures the Gaussian has Area = 1.0
# This matches the Frequency domain filter where Amplitude = 1.0 at w=0.
norm_factor = 1.0 / (sigma * np.sqrt(2 * np.pi))

for t in shifts:
    # Create a local integration window centered on t
    # +/- 4 sigma captures 99.9% of the Gaussian energy
    x_local = np.linspace(t - 4*sigma, t + 4*sigma, 500)
    
    # Evaluate signal and kernel on this local window
    sig_local = get_signal_val(x_local)
    k_local = np.exp(-0.5 * ((x_local - t) / sigma)**2) * norm_factor
    
    # Integrate
    val = np.trapz(sig_local * k_local, x_local)
    conv_vals.append(val)

conv_vals = np.array(conv_vals)

# Note: We do NOT normalize conv_vals to [0,1] anymore. 
# We want to see the actual amplitude to match the Fourier reconstruction.

# 3. Setup Animation Figure
fig_anim = plt.figure(figsize=(8,6))
ax1 = fig_anim.add_axes([0.05, 0.55, 0.9, 0.4])
ax2 = fig_anim.add_axes([0.05, 0.08, 0.9, 0.4])

for ax in (ax1, ax2):
    ax.set_axis_off()

# Ax1: Input Signal + Sliding Kernel
ax1.set_xlim(x.min(), x.max())
ax1.set_ylim(-2.5, 2.5)
ax1.plot(x, sum_signal, linewidth=LINEWIDTH*1.0, color=BLACK, solid_capstyle='round', alpha=0.9)

# Ax2: Convolution Result
ax2.set_xlim(x.min(), x.max())
# Match limits to the signal amplitude so we can compare input vs output sizes
ax2.set_ylim(-2.5, 2.5) 

# Graphical elements to update
kernel_line, = ax1.plot([], [], linewidth=LINEWIDTH, color='#333333', alpha=0.8)
conv_line, = ax2.plot([], [], linewidth=LINEWIDTH*1.1, color=BLACK, solid_capstyle='round', alpha=1.0)

# Reference line for zero
ax2.axhline(0, color='gray', alpha=0.3, linewidth=1, linestyle='--')

def init():
    kernel_line.set_data([], [])
    conv_line.set_data([], [])
    return kernel_line, conv_line

def update(i):
    t = shifts[i]
    
    # Visual Kernel: We scale it just for visibility on the plot (it's not the integration kernel)
    # Using a peak of 1.5 looks good against the signal amplitude
    vis_scale = 1.5
    k_vis = np.exp(-0.5 * ((x - t) / sigma)**2) * vis_scale
    kernel_line.set_data(x, k_vis)
    
    # Update convolution curve
    conv_line.set_data(shifts[:i+1], conv_vals[:i+1])
    return kernel_line, conv_line

anim = FuncAnimation(fig_anim, update, frames=len(shifts), init_func=init, blit=True, interval=50)

# Save
writer = FFMpegWriter(
    fps=60,
    codec='libx265',
    extra_args=['-pix_fmt', 'yuv420p', '-crf', '18', '-preset', 'slow', '-tag:v', 'hvc1']
)
anim.save(str(export_dir / 'convolution_animation.mov'), writer=writer, dpi=300)
plt.close(fig_anim)

In [ ]:
# %%
# ---------------- Animation: Split into two videos (Part 1 & Part 2) ----------------

# Common settings
# Use the same exact figsize as your static images so they align perfectly in Keynote
VIDEO_FIGSIZE = (10, 3.5) 
fps = 60

# We assume 'shifts', 'x', 'sigma', 'sum_signal', and 'conv_vals' are already 
# calculated from your previous code block.

from matplotlib.animation import FFMpegWriter

# --- Video 1: The Input Signal + Sliding Kernel ---
fig1, ax1 = plt.subplots(figsize=VIDEO_FIGSIZE)
ax1.set_axis_off()
ax1.set_xlim(x.min(), x.max())
ax1.set_ylim(-2.5, 2.5)

# Static background: Signal
ax1.plot(x, sum_signal, linewidth=LINEWIDTH*1.0, color=BLACK, solid_capstyle='round', alpha=0.9)
# Dynamic part: Kernel
kernel_line, = ax1.plot([], [], linewidth=LINEWIDTH, color='#333333', alpha=0.8)

def init1():
    kernel_line.set_data([], [])
    return (kernel_line,)

def update1(i):
    t = shifts[i]
    # Visual kernel (scaled for visibility)
    vis_scale = 1.5
    k_vis = np.exp(-0.5 * ((x - t) / sigma)**2) * vis_scale
    kernel_line.set_data(x, k_vis)
    return (kernel_line,)

anim1 = FuncAnimation(fig1, update1, frames=len(shifts), init_func=init1, blit=True, interval=50)

# Save Video 1
writer = FFMpegWriter(fps=fps, codec='libx265', extra_args=['-pix_fmt', 'yuv420p', '-crf', '18', '-tag:v', 'hvc1'])
print("Saving Part 1...")
anim1.save(str(export_dir / 'convolution_part1_input.mov'), writer=writer, dpi=300)
plt.close(fig1)


# --- Video 2: The Convolution Result ---
fig2, ax2 = plt.subplots(figsize=VIDEO_FIGSIZE)
ax2.set_axis_off()
ax2.set_xlim(x.min(), x.max())
ax2.set_ylim(-2.5, 2.5) # Matches Part 1 vertical scale

# Dynamic part: Convolution curve
conv_line, = ax2.plot([], [], linewidth=LINEWIDTH*1.1, color=BLACK, solid_capstyle='round', alpha=1.0)
# Optional: Reference line
ax2.axhline(0, color='gray', alpha=0.3, linewidth=1, linestyle='--')

def init2():
    conv_line.set_data([], [])
    return (conv_line,)

def update2(i):
    # Update convolution curve up to current frame
    conv_line.set_data(shifts[:i+1], conv_vals[:i+1])
    return (conv_line,)

anim2 = FuncAnimation(fig2, update2, frames=len(shifts), init_func=init2, blit=True, interval=50)

# Save Video 2
print("Saving Part 2...")
anim2.save(str(export_dir / 'convolution_part2_output.mov'), writer=writer, dpi=300)
plt.close(fig2)

print("Done! Videos saved to:", export_dir)

In [ ]:
# Print file locations for downloading
paths = {
    'signal_components': str(path1),
    'spectrum_diracs': str(path2),
    'gaussian_spatial': str(path3),
    'gaussian_frequency': str(path4),
    'attenuated_diracs': str(path5),
    'filtered_signal': str(path6),
    # 'convolution_animation': str(gif_path)
}
paths